In [ ]:
import pandas as pd


data = pd.read_csv('task.csv')

data = data.dropna()

voltage_outliers_bool = data['voltage'].lt(2.5) | data['voltage'].gt(4.5)
number_of_voltage_outliers = voltage_outliers_bool.sum()
print('Number of voltage outliers:', number_of_voltage_outliers)

current_outliers_bool = data['current'].lt(-100) | data['current'].gt(100)
number_of_current_outliers = current_outliers_bool.sum()
print('Number of current outliers:', number_of_current_outliers)

data = data[~data.index.isin(voltage_outliers_bool.index)]
data = data[~data.index.isin(current_outliers_bool.index)]

data['charge_current'] = data['current'].diff()

if len(data['soc']) > 0:
   
    fECs = []
    previous_SOC = data['soc'].iloc[0]
    for i in range(1, len(data)):
        current_SOC = data['soc'].iloc[i]
        if previous_SOC < current_SOC:
           
            if data['charge_current'].iloc[i] > 0:
                fECs.append(data['FEC'].iloc[i])
        else:
           
            if data['charge_current'].iloc[i] < 0:
                fECs.append(data['FEC'].iloc[i])

        previous_SOC = current_SOC

    print('FECs:', fECs)


    capacity = []
    for i in range(10):
        start_index = i * (len(data)//10)
        end_index = start_index + (len(data)//10)
        current_data = data.iloc[start_index:end_index]

        average_charge_current = abs(current_data['charge_current'].mean())

        capacity_i = (current_data['timestamp'].iloc[-1] - current_data['timestamp'].iloc[0]) * average_charge_current

        capacity.append(capacity_i)

    print('Average capacity over the first ten cycles:', sum(capacity) / len(capacity))
else:
    print('There are no SOC values to analyze.')
